# Full Data Calculation for Weapons

## Setting Up

In [1]:
import math
import pandas as pd
from scripts.weapon import max_shots, burst, penetrating_damage, hits_to_kill, chance_to_damage
from scripts.accuracy import average_to_hit

aliens = pd.read_csv("datasets/aliens.csv")
weapon_stats = pd.read_csv("datasets/weapon_stats.csv")
ammo_stats = pd.read_csv("datasets/ammo_stats.csv")
weapon_firemodes = pd.read_csv("datasets/weapon_firemodes.csv")
damage_modifiers = pd.read_csv("datasets/damage_modifiers.csv")

## Data Merge

In [2]:
weapon_data = weapon_stats.merge(ammo_stats, on="weapon")
weapon_data = weapon_data.merge(weapon_firemodes, on="weapon")

## Chance to Hit

We will use an average Accuracy of 50. Meaning the soldier fails half of the time.

The chance of hit formula is simple. Just multiply the soldier's accuracy chance by the weapon's accuracy chance. So an average of 50 to a weapon of Accuracy 90 means 0.50 * 0.9 = 0.45 chance of hit.

In [3]:
weapon_data["average_chance_to_hit"] = weapon_data.apply(lambda x: average_to_hit(x["accuracy"]), axis=1)

## Shots

In [4]:
weapon_data["burst"] = weapon_data.apply(burst, axis=1)
weapon_data["turn_max_shots"] = weapon_data.apply(max_shots, axis=1)
weapon_data["turn_max_shots"] = weapon_data.apply(max_shots, axis=1)
weapon_data["turns_to_reload"] = weapon_data.apply(lambda x: x["capacity"] / x["turn_max_shots"], axis=1)
weapon_data['turns_to_reload'] = weapon_data['turns_to_reload'].fillna(0)
weapon_data["turns_to_reload"] = weapon_data.apply(lambda x: x["turns_to_reload"], axis=1)

## Damage Range

In [5]:
weapon_data["turn_damage_max"] = weapon_data.apply(lambda x: x["turn_max_shots"] * x["damage"], axis=1)
weapon_data["turn_damage_expected"] = weapon_data.apply(lambda x: x["turn_damage_max"] * x["average_chance_to_hit"], axis=1)
weapon_data["clip_damage"] = weapon_data.apply(lambda x: x["capacity"] * x["damage"], axis=1)

## Costs

In [6]:
weapon_data["damage_per_time_unit"] = weapon_data.apply(lambda x: x["damage"] / x["time_units"], axis=1)

## Damage to Target Type

### Data Merge

In [7]:
weapon_data_targets = weapon_data.append([weapon_data] * (len(damage_modifiers.index)-1), ignore_index=True)
weapon_data_targets["key"] = weapon_data_targets.groupby(["weapon", "ammo", "fire_mode"]).cumcount()

targets = damage_modifiers["target"]

weapon_data_targets = pd.merge(weapon_data_targets, targets, left_on="key", right_index=True, how="left", sort=False)
weapon_data_targets = weapon_data_targets.drop(columns=["key"])

### Damage vs Type

In [8]:
def damage_modifier(group):
    """
    Returns the damage modifier for the weapon target.
    """
    target = damage_modifiers[damage_modifiers["target"] == group["target"]]
    damage = target[group["damage_type"]].values[0]
    return damage / 100

In [9]:
weapon_data_targets["target_damage"] = weapon_data_targets.apply(lambda x: x["damage"] * damage_modifier(x), axis=1)
weapon_data_targets["turn_target_damage_max"] = weapon_data_targets.apply(lambda x: x["turn_max_shots"] * x["target_damage"], axis=1)
weapon_data_targets["turn_target_damage_expected"] = weapon_data_targets.apply(lambda x: x["turn_target_damage_max"] * x["average_chance_to_hit"], axis=1)

### Costs

In [10]:
weapon_data_targets["target_damage_per_time_unit"] = weapon_data_targets.apply(lambda x: x["target_damage"] / x["time_units"], axis=1)

## Damage to Alien

### Average Armor

In [11]:
aliens_armor = aliens[["alien", "armor_front", "armor_side", "armor_rear", "armor_under"]]
aliens_armor = aliens_armor.set_index("alien")
aliens_armor = aliens_armor.mean(axis=1)
aliens_armor = aliens_armor.reset_index()
aliens_armor = aliens_armor.rename(columns={0: "armor"})

### Base data

In [12]:
weapon_data_aliens = aliens.merge(aliens_armor, on="alien")
weapon_data_aliens = weapon_data_aliens.merge(weapon_data_targets, on="target")
weapon_data_aliens = weapon_data_aliens.rename(columns={"time_units_x": "time_units"})
weapon_data_aliens = weapon_data_aliens.rename(columns={"time_units_y": "weapon_time_units"})

### Penetrating Damage

In [13]:
weapon_data_aliens["chance_to_penetrate"] = weapon_data_aliens.apply(lambda x: chance_to_damage(x["target_damage"], x["armor"]), axis=1)
weapon_data_aliens["chance_to_penetrate"] = pd.to_numeric(weapon_data_aliens["chance_to_penetrate"])
weapon_data_aliens["chance_of_hit_and_penetrate"] = weapon_data_aliens.apply(lambda x: x["average_chance_to_hit"] * x["chance_to_penetrate"], axis=1)

In [14]:
weapon_data_aliens["penetrating_damage"] = weapon_data_aliens.apply(lambda x: penetrating_damage(x["target_damage"],x["armor"]), axis=1)
weapon_data_aliens["turn_penetrating_damage_max"] = weapon_data_aliens.apply(lambda x: x["turn_max_shots"] * x["penetrating_damage"], axis=1)
weapon_data_aliens["turn_penetrating_damage_expected"] = weapon_data_aliens.apply(lambda x: x["turn_penetrating_damage_max"] * x["average_chance_to_hit"], axis=1)
weapon_data_aliens["clip_penetrating_damage"] = weapon_data_aliens.apply(lambda x: x["capacity"] * x["penetrating_damage"], axis=1)

### Hits to Kill

In [15]:
weapon_data_aliens["hits_to_kill"] = weapon_data_aliens.apply(lambda x: hits_to_kill(x["penetrating_damage"], x["health"]), axis=1)
weapon_data_aliens["turn_expected_kills"] = weapon_data_aliens.apply(lambda x: x["turn_max_shots"] / x["hits_to_kill"], axis=1)

In [16]:
weapon_data_aliens["overkill"] = weapon_data_aliens.apply(lambda x: (x["hits_to_kill"] * x["penetrating_damage"]) - x["health"], axis=1)

### Costs

In [17]:
weapon_data_aliens["penetrating_damage_per_time_unit"] = weapon_data_aliens.apply(lambda x: x["penetrating_damage"] / x["weapon_time_units"], axis=1)
weapon_data_aliens["overkill_per_time_unit"] = weapon_data_aliens.apply(lambda x: x["overkill"] / x["weapon_time_units"], axis=1)

## Data Generated

In [18]:
weapon_data[weapon_data["weapon"]=="heavy_plasma"]

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,burst,turn_max_shots,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit


In [19]:
weapon_data.head()

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,burst,turn_max_shots,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit
0,dart_gun,1,dart_gun_clip,16,armor_piercing,12.0,snapshot,20,40,0.220,1,3,4.0,48,10.56,192.0,0.800000
1,dart_gun,1,dart_gun_clip,16,armor_piercing,12.0,aimed,50,80,0.440,1,1,12.0,16,7.04,192.0,0.320000
2,jet_harpoon,2,jet_harpoon_clip,10,armor_piercing,32.0,auto,40,40,0.220,3,2,16.0,20,4.40,320.0,0.250000
3,jet_harpoon,2,jet_harpoon_clip,10,armor_piercing,32.0,snapshot,35,60,0.330,1,2,16.0,20,6.60,320.0,0.285714
4,jet_harpoon,2,jet_harpoon_clip,10,armor_piercing,32.0,aimed,70,90,0.495,1,1,32.0,10,4.95,320.0,0.142857


In [20]:
weapon_data_targets.head()

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,...,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit,target,target_damage,turn_target_damage_max,turn_target_damage_expected,target_damage_per_time_unit
0,dart_gun,1,dart_gun_clip,16,armor_piercing,12.0,snapshot,20,40,0.220,...,4.0,48,10.56,192.0,0.800000,terrain,16.0,48.0,10.56,0.800000
1,dart_gun,1,dart_gun_clip,16,armor_piercing,12.0,aimed,50,80,0.440,...,12.0,16,7.04,192.0,0.320000,terrain,16.0,16.0,7.04,0.320000
2,jet_harpoon,2,jet_harpoon_clip,10,armor_piercing,32.0,auto,40,40,0.220,...,16.0,20,4.40,320.0,0.250000,terrain,10.0,20.0,4.40,0.250000
3,jet_harpoon,2,jet_harpoon_clip,10,armor_piercing,32.0,snapshot,35,60,0.330,...,16.0,20,6.60,320.0,0.285714,terrain,10.0,20.0,6.60,0.285714
4,jet_harpoon,2,jet_harpoon_clip,10,armor_piercing,32.0,aimed,70,90,0.495,...,32.0,10,4.95,320.0,0.142857,terrain,10.0,10.0,4.95,0.142857


In [21]:
weapon_data_aliens.head()

,alien,time_units,stamina,health,bravery,reactions,firing_accuracy,throwing_accuracy,strength,psionic_strength,...,chance_of_hit_and_penetrate,penetrating_damage,turn_penetrating_damage_max,turn_penetrating_damage_expected,clip_penetrating_damage,hits_to_kill,turn_expected_kills,overkill,penetrating_damage_per_time_unit,overkill_per_time_unit
0,Aquatoid Soldier,54,90,30,80,63,52,58,30,35,...,0.220,16.0,48.0,10.56,192.0,2.0,1.500000,2.0,0.800000,0.10
1,Aquatoid Soldier,54,90,30,80,63,52,58,30,35,...,0.440,16.0,16.0,7.04,192.0,2.0,0.500000,2.0,0.320000,0.04
2,Aquatoid Soldier,54,90,30,80,63,52,58,30,35,...,0.220,10.0,20.0,4.40,320.0,3.0,0.666667,0.0,0.250000,0.00
3,Aquatoid Soldier,54,90,30,80,63,52,58,30,35,...,0.330,10.0,20.0,6.60,320.0,3.0,0.666667,0.0,0.285714,0.00
4,Aquatoid Soldier,54,90,30,80,63,52,58,30,35,...,0.495,10.0,10.0,4.95,320.0,3.0,0.333333,0.0,0.142857,0.00
